Production planning problem from book Operations Research - A Model-Baesed Approach 3rd Edition by H.A Eiselt and Carl-Louis Sandblom, chapter 2.2.1, page 20


In [9]:
import linopy as lp
import pandas as pd
import xarray as xr

products = pd.Index(['P1', 'P2', 'P3'], name='Products')
machines = pd.Index(['M1', 'M2'], name='Machines')

processing_times = xr.DataArray([[3, 5, 4], [6, 1, 3]], coords=[machines, products])

processing_times

<xarray.DataArray (Machines: 2, Products: 3)> Size: 48B
array([[3, 5, 4],
       [6, 1, 3]])
Coordinates:
  * Machines  (Machines) object 16B 'M1' 'M2'
  * Products  (Products) object 24B 'P1' 'P2' 'P3'

In [10]:
machines_availability = xr.DataArray([540, 480], coords=[machines])

In [11]:
machines_availability

<xarray.DataArray (Machines: 2)> Size: 16B
array([540, 480])
Coordinates:
  * Machines  (Machines) object 16B 'M1' 'M2'

In [12]:
products_profit = xr.DataArray([5, 3.5, 4.5], coords=[products])
products_profit

<xarray.DataArray (Products: 3)> Size: 24B
array([5. , 3.5, 4.5])
Coordinates:
  * Products  (Products) object 24B 'P1' 'P2' 'P3'

In [13]:
linopy_model = lp.Model()

production = linopy_model.add_variables(lower=0, coords=[products], name='x')

machine_usage = production * processing_times
linopy_model.add_constraints(machine_usage.sum('Products') <= machines_availability)

total_profit_per_product = products_profit * production
linopy_model.add_objective(total_profit_per_product.sum(), sense='max')
linopy_model

Linopy LP model

Variables:
----------
 * x (Products)

Constraints:
------------
 * con0 (Machines)

Status:
-------
initialized

In [14]:
linopy_model.solve(solver='highs')

Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-08o1p28q has 2 rows; 3 cols; 6 nonzeros
Coefficient ranges:
  Matrix [1e+00, 6e+00]
  Cost   [4e+00, 5e+00]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 5e+02]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 3(-0); elements 6(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.1249939713e+00 Ph1: 2(7); Du: 3(4.12499) 0s
          2     6.4000000000e+02 Pr: 0(0) 0s
Model name          : linopy-problem-08o1p28q
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  6.4000000000e+02
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00
Writing the solution to /private/var/folders/y9/wlhs0yq90s3bmc43pnxzsl_40000gp/T/linopy-solve-wdwh51g_.sol


('ok', 'optimal')

In [15]:
linopy_model.solution

<xarray.Dataset> Size: 48B
Dimensions:   (Products: 3)
Coordinates:
  * Products  (Products) object 24B 'P1' 'P2' 'P3'
Data variables:
    x         (Products) float64 24B 20.0 0.0 120.0